In [71]:
import numpy as np
import os
import matplotlib.pyplot as plt

os.environ['CUDA_VISIBLE_DEVICES'] = '1'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [72]:
# !pip install tensorflow-gpu==2.3
# !pip install sklearn

In [73]:
import tensorflow
import tensorflow as tf

In [74]:
tf.config.list_physical_devices('GPU')

[]

In [75]:
tf.test.is_gpu_available()

False

In [76]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 12344401564678218446]

In [77]:
actions = [
     '1slap-L',
     '1slap-R',
     '2pushing-A',
     '2pushing-L',
     '2pushing-R',
     '3punching-L',
     '3puncning-R',
     '4kicking-L',
     '4kicking-R',
     '5tread-L',
     '5tread-R',
     '6footpushing-L',
     '6footpushing-R',
     'normal']

file_dir = 'C:/Users/chltp/detect_video/dataset/'#seq 파일을 dataset 폴더에 넣어주세요

npy_file = []
for name in os.listdir(file_dir):
    if not len(np.load(file_dir + name)) == 0:
        npy_file.append(np.load(file_dir + name))

data = np.concatenate(npy_file, axis=0)

data.shape

(3389, 30, 67)

In [89]:
x_data = data[:, :, :]
labels = data[:, 0, -1]

print(x_data.shape)
print(labels.shape)

(3389, 30, 67)
(3389,)


In [ ]:
for i in range(3389):
    for j in range(29):
        if x_data[i][j][67]=='7' or x_data[i][j][67]=='9' or x_data[i][j][67]=='11':
            x_data[i][j][67]='1'
        elif x_data[i][j][67]='0'

In [ ]:
# punching = []
# kicking = []
# normal = []

slap_L = []
slap_R = []
pushing_A = []
pushing_L = []
pushing_R = []
punching_L = []
puncning_R = []
kicking_L = []
kicking_R = []
tread_L = []
tread_R = []
footpushing_L = []
footpushing_R = []
normal = []

count = 0
for count in range(len(x_data)):
    temp_count = 0
    temp = 0
    for j in range(30):
        for i in range(54, 66):
            t = data[count][j][i]
    #         print(t)
            if t != 0 and not np.isnan(t):
                temp += int(t)
                temp_count += 1
    #             print(temp)
        if not temp_count == 0:
            if data[count][j][66] == 0:
                slap_L.append(temp / temp_count)
            if data[count][j][66] == 1:
                slap_R.append(temp / temp_count)
            if data[count][j][66] == 2:
                pushing_A.append(temp / temp_count)
            if data[count][j][66] == 3:
                pushing_L.append(temp / temp_count)
            if data[count][j][66] == 4:
                pushing_R.append(temp / temp_count)
            if data[count][j][66] == 5:
                punching_L.append(temp / temp_count)
            if data[count][j][66] == 6:
                puncning_R.append(temp / temp_count)
            if data[count][j][66] == 7:
                kicking_L.append(temp / temp_count)
            if data[count][j][66] == 8:
                kicking_R.append(temp / temp_count)
            if data[count][j][66] == 9:
                tread_L.append(temp / temp_count)
            if data[count][j][66] == 10:
                tread_R.append(temp / temp_count)
            if data[count][j][66] == 11:
                footpushing_L.append(temp / temp_count)
            if data[count][j][66] == 12:
                footpushing_R.append(temp / temp_count)
            if data[count][j][66] == 13 and len(normal) < 40000:
                normal.append(temp / temp_count)
            
plt.plot(slap_L, '--', slap_R, '--', pushing_A, '--', pushing_L, '--', pushing_R, '--', punching_L, '--', puncning_R, '--', kicking_L, '--', kicking_R, '--', tread_L, '--', tread_R, '--', footpushing_L, '--', footpushing_R, '--', normal, 'r--')
plt.show()

In [ ]:
plt.plot(slap_L, 'r--', slap_R, 'r--', pushing_A, 'r--', pushing_L, 'r--', pushing_R, 'r--', punching_L, 'r--', puncning_R, 'r--', kicking_L, 'g--', kicking_R, 'g--', tread_L, 'g--', tread_R, 'g--', footpushing_L, 'g--', footpushing_R, 'g--', normal, 'b--')
plt.show()

In [ ]:
from tensorflow.keras.utils import to_categorical

y_data = to_categorical(labels, num_classes=len(actions))
y_data.shape

In [ ]:
y_data[0]

In [ ]:
array([0., 1.],

In [ ]:
from sklearn.model_selection import train_test_split


x_data = x_data.astype(np.float32)
y_data = y_data.astype(np.float32)

x_train, x_val, y_train, y_val = train_test_split(x_data, y_data, test_size=0.1, random_state=2021)

print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

model = Sequential([
    LSTM(64, activation='relu', input_shape=x_train.shape[1:3]),
    Dense(32, activation='relu'),
    Dense(len(actions), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
model.summary()

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

history = model.fit(
    x_train,
    y_train,
    validation_data=(x_val, y_val),
    epochs=100,
    callbacks=[
        ModelCheckpoint('models/model.h5', monitor='val_acc', verbose=1, save_best_only=True, mode='auto'),
        ReduceLROnPlateau(monitor='val_acc', factor=0.5, patience=50, verbose=1, mode='auto')
    ]
)

In [ ]:
import matplotlib.pyplot as plt

fig, loss_ax = plt.subplots(figsize=(16, 10))
acc_ax = loss_ax.twinx()

loss_ax.plot(history.history['loss'], 'y', label='train loss')
loss_ax.plot(history.history['val_loss'], 'r', label='val loss')
loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
loss_ax.legend(loc='upper left')

acc_ax.plot(history.history['acc'], 'b', label='train acc')
acc_ax.plot(history.history['val_acc'], 'g', label='val acc')
acc_ax.set_ylabel('accuracy')
acc_ax.legend(loc='upper left')

plt.show()

In [ ]:
from sklearn.metrics import multilabel_confusion_matrix
from tensorflow.keras.models import load_model

model = load_model('models/model.h5')

y_pred = model.predict(x_val)

multilabel_confusion_matrix(np.argmax(y_val, axis=1), np.argmax(y_pred, axis=1))